# Dockerize application

## Prepare docker

Write docker ingore

In [15]:
%%writefile .dockerignore

.ipynb_checkpoints
__pycache__
.venv
.venv_test
chatglm-6b

Overwriting .dockerignore


Write Dockerfile to build docker

In [60]:
%%writefile Dockerfile

FROM nvidia/cuda:12.2.0-devel-ubuntu22.04 AS base

## install python and pip
RUN apt-get update -y
RUN apt-get install --assume-yes software-properties-common
RUN apt-get install python3-pip -y
RUN apt-get install vim -y

## create user instead root
RUN adduser --disabled-login --gecos "First Last,RoomNumber,WorkPhone,HomePhone" llm
USER llm
WORKDIR /home/llm
ENV PATH="/home/llm/.local/bin:${PATH}"

# update pip
RUN pip3 install --upgrade pip -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
# COPY chatglm2-6b chatglm-6b and MAKE the following files cachable
COPY chatglm2-6b/config.json chatglm-6b/config.json
COPY chatglm2-6b/configuration_chatglm.py chatglm-6b/configuration_chatglm.py
COPY chatglm2-6b/modeling_chatglm.py chatglm-6b/modeling_chatglm.py
COPY chatglm2-6b/MODEL_LICENSE chatglm-6b/MODEL_LICENSE
COPY chatglm2-6b/pytorch_model-00001-of-00007.bin chatglm-6b/pytorch_model-00001-of-00007.bin
COPY chatglm2-6b/pytorch_model-00002-of-00007.bin chatglm-6b/pytorch_model-00002-of-00007.bin
COPY chatglm2-6b/pytorch_model-00003-of-00007.bin chatglm-6b/pytorch_model-00003-of-00007.bin
COPY chatglm2-6b/pytorch_model-00004-of-00007.bin chatglm-6b/pytorch_model-00004-of-00007.bin
COPY chatglm2-6b/pytorch_model-00005-of-00007.bin chatglm-6b/pytorch_model-00005-of-00007.bin
COPY chatglm2-6b/pytorch_model-00006-of-00007.bin chatglm-6b/pytorch_model-00006-of-00007.bin
COPY chatglm2-6b/pytorch_model-00007-of-00007.bin chatglm-6b/pytorch_model-00007-of-00007.bin
COPY chatglm2-6b/pytorch_model.bin.index.json chatglm-6b/pytorch_model.bin.index.json
COPY chatglm2-6b/quantization.py chatglm-6b/quantization.py
COPY chatglm2-6b/README.md chatglm-6b/README.md
COPY chatglm2-6b/tokenization_chatglm.py chatglm-6b/tokenization_chatglm.py
COPY chatglm2-6b/tokenizer_config.json chatglm-6b/tokenizer_config.json
COPY chatglm2-6b/tokenizer.model chatglm-6b/tokenizer.model


FROM base AS llm_smart_search_python_env
## copy requirements and install dependencies
COPY --chown=llm:llm requirements.txt .
RUN pip3 install --ignore-installed --no-cache-dir --user -r requirements.txt -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

FROM llm_smart_search_python_env AS llm_smart_search
COPY --chown=llm:llm main.py .

## run the application
CMD [ "flask", "--app", "main", "run","--host","0.0.0.0","--port","5000"]


Overwriting Dockerfile


## Build docker
To build the docker image, first login to ECR.

### Build image

Next, build your custom docker container, tagging it with the name "llm_smart_search".

In [61]:
%%sh
docker build -t llm_smart_search --target llm_smart_search -f Dockerfile . --build-arg region=$(aws configure get region)

Sending build context to Docker daemon  24.98GB
Step 1/31 : FROM nvidia/cuda:12.2.0-devel-ubuntu22.04
 ---> c5a448dd0547
Step 2/31 : RUN apt-get update -y
 ---> Using cache
 ---> 81dbe3b14244
Step 3/31 : RUN apt-get install --assume-yes software-properties-common
 ---> Using cache
 ---> 481b61e472b0
Step 4/31 : RUN apt-get install vim -y
 ---> Using cache
 ---> 8a63e2c609c0
Step 5/31 : RUN apt-get install python3-pip -y
 ---> Using cache
 ---> 610e4e975059
Step 6/31 : RUN adduser --disabled-login --gecos "First Last,RoomNumber,WorkPhone,HomePhone" llm
 ---> Using cache
 ---> d63e12df40b2
Step 7/31 : USER llm
 ---> Using cache
 ---> 7dfe6fa60682
Step 8/31 : WORKDIR /home/llm
 ---> Using cache
 ---> 2ccc74ca77d9
Step 9/31 : ENV PATH="/home/llm/.local/bin:${PATH}"
 ---> Using cache
 ---> 18c6143c95ce
Step 10/31 : RUN pip3 install --upgrade pip -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
 ---> Using cache
 ---> 96caae63e59f
Step 11/31 : COPY chatglm2-6b/conf

### Push image to ecr
Login to AWS ECR

In [ ]:
%%sh
REGION=$(aws configure get region)
account=$(aws sts get-caller-identity --query Account --output text)

# https://docs.aws.amazon.com/sagemaker/latest/dg-ecr-paths/ecr-cn-north-1.html
docker login --username AWS --password $(aws ecr get-login-password --region cn-north-1) 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn
aws ecr get-login-password --region ${REGION} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${REGION}.amazonaws.com.cn

In [ ]:
!pip install -q boto3

Create ecr repo

In [49]:
import boto3
import sagemaker

# role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

region = sagemaker_session.boto_region_name
account = !aws sts get-caller-identity --query Account --output text
algorithm_name = "llm_smart_search"

ecr_image = '{}.dkr.ecr.{}.amazonaws.com.cn/{}:latest'.format(account[0], region, algorithm_name)
print('ecr_image: {}'.format(ecr_image))

ecr_client = boto3.client('ecr')
try:
    response = ecr_client.describe_repositories(
        repositoryNames=[
            algorithm_name,
        ],
    )
    print("Repo exists...")
except Exception as e:
    create_repo = ecr_client.create_repository(repositoryName=algorithm_name)
    print("Created repo...")


ecr_image: 592356948544.dkr.ecr.cn-north-1.amazonaws.com.cn/llm_smart_search:latest
Repo exists...


Push the image

In [50]:
!docker tag {algorithm_name} {ecr_image}
!docker push {ecr_image}

The push refers to repository [592356948544.dkr.ecr.cn-north-1.amazonaws.com.cn/llm_smart_search]

5a193b37: Preparing 
34f2e81f: Preparing 
da7e0b0c: Preparing 
91cad399: Preparing 
e3a983ea: Preparing 
7c06f7c3: Preparing 
63807a65: Preparing 
d7e2ba9f: Preparing 
63ddf0a0: Preparing 
2db41638: Preparing 
0fd8f4d5: Preparing 
0b80bc94: Preparing 
16ad47c1: Preparing 
d220aee8: Preparing 
d7e2ba9f: Waiting g 
63ddf0a0: Waiting g 
0fd8f4d5: Waiting g 
1a096ada: Preparing 
bc7e829b: Preparing 
0820f4d6: Preparing 
daf64309: Preparing 
fe055fc5: Preparing 
cc35140f: Preparing 
6ad47c1: Waiting g 
1ed71ae3: Preparing 
220aee8: Waiting g 
c1c8a611: Preparing 
685e426: Waiting g 
cc35140f: Waiting g 
1ac317ad: Waiting g 
183ab665: Preparing 
9b4c9dc9: Preparing 
fa8b3ec4: Preparing 
ed71ae3: Waiting g 
7ce4f321: Preparing 
1c8a611: Waiting g 
a193b37: Pushed   4.786GB/4.758GBPushing  398.3MB/1.053GBPushing  553.7MB/1.815GBPushing  918.3MB/4.758GBPushing  270.7MB/1.927GBPushing  1.347GB/1.92